<a href="https://colab.research.google.com/github/neilgautam/Association-Rule-Mining/blob/master/OnOff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from matplotlib import pyplot
from sklearn.externals import joblib
import numpy as np
import datetime
import pickle

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path= "./drive/My Drive/uk dale dataset/channel_{}.dat"

In [0]:
def string_to_timestamp(string):
    date_time_obj = datetime.datetime.strptime(string, '%Y-%m-%d %H:%M:%S')
    timestamp = date_time_obj.timestamp()
    return timestamp

In [0]:
class data_prep():
    def __init__(self, path):
      self.df = pd.read_csv(path, sep='\s+', header=None)

    def resampler(self,time = "5min"):
      dataframe = self.df.set_index([0])
      dataframe.index = pd.to_datetime(dataframe.index,unit = "s")
      resample = dataframe.resample(time)
      self.resampled_data = resample.sum()
      self.resampled_data = self.resampled_data/300

    def get_resampled_with_timestamp(self):
      self.final_data = self.resampled_data.reset_index()

    def get_final_data(self):
      self.resampler()
      self.get_resampled_with_timestamp()
      return self.final_data

In [6]:
for i in range(1,54):
    try:
        prep = data_prep("./drive/My Drive/uk dale dataset/channel_{}.dat".format(i))
        print("File {} data_prep , Initialised".format(i))
        data = prep.get_final_data()
        print(data.head(5))
        joblib.dump(data,"./drive/My Drive/uk dale dataset/on_off_files/onoff_{}.pkl".format(i))
        print("File Saved>>>>")
    except:
        continue    

File 1 data_prep , Initialised
                    0          1
0 2012-11-09 22:25:00  27.386667
1 2012-11-09 22:30:00  88.003333
2 2012-11-09 22:35:00  92.173333
3 2012-11-09 22:40:00  56.000000
4 2012-11-09 22:45:00  53.633333
File Saved>>>>
File 2 data_prep , Initialised
                    0         1
0 2012-11-09 22:25:00  1.363333
1 2012-11-09 22:30:00  3.216667
2 2012-11-09 22:35:00  3.223333
3 2012-11-09 22:40:00  3.056667
4 2012-11-09 22:45:00  1.266667
File Saved>>>>
File 3 data_prep , Initialised
                    0    1
0 2012-11-09 22:25:00  0.0
1 2012-11-09 22:30:00  0.0
2 2012-11-09 22:35:00  0.0
3 2012-11-09 22:40:00  0.0
4 2012-11-09 22:45:00  0.0
File Saved>>>>
File 5 data_prep , Initialised
                    0    1
0 2012-11-09 22:25:00  0.0
1 2012-11-09 22:30:00  0.0
2 2012-11-09 22:35:00  0.0
3 2012-11-09 22:40:00  0.0
4 2012-11-09 22:45:00  0.0
File Saved>>>>
File 6 data_prep , Initialised
                    0         1
0 2012-11-09 22:25:00  0.056667
1 2012-

In [0]:
class to_serialised_on_off():
    def __init__(self, data):
      self.data = data
      self.min_ = 0
      self.avg_ = 0
      self.max_ = 0
      self.find_min_max(flag = 1)

    def append_to_df(self, val):
      self.data.append(val)
      self.find_min_max()

    def find_min_max(self, flag = 0):
      if(flag == 0):
        val = self.data.loc[-1][-1]
        if( val > self.max_ ):
          self.max_ = val
        elif( val < self.min_ ):
          self.min_ = val
        self.avg_ += val/self.data.shape[0]
      elif(flag):
        self.max_ = max(self.data.loc[:][1])
        self.min_ = min(self.data.loc[:][1])
        self.avg_ = np.mean(self.data.loc[:][1].values)

      self.calc_thresh()

    def calc_thresh(self):
      self.thresh = (self.max_ - self.min_)/self.avg_

    def on_off(self):
      of = []
      for i in range(self.data.shape[0]):
        if(self.data.loc[i][1] > self.thresh):
          of.append(str(self.data.loc[i][0]))
        else:
          of.append(0)

      return of

In [0]:
for i in range(1,54):
    data = joblib.load("./drive/My Drive/uk dale dataset/on_off_files/onoff_{}.pkl".format(i))
    print("File {} Loaded".format(i))
    print(data.shape)
    try: 
        obj1 = to_serialised_on_off(data[:107136])
    except:
        obj1 = to_serialised_on_off(data) 
    print(obj1.thresh, obj1.max_, obj1.min_, obj1.avg_)
    arr = obj1.on_off()
    print("File Processed")
    joblib.dump(arr,"./drive/My Drive/uk dale dataset/ONOFF_timefiles/on_off_time_{}.pkl".format(i))
    print("file_saved>>>")